# YOLOv26 Quantization Aware Training (QAT)

## Overview
This notebook implements a complete Quantization Aware Training (QAT) pipeline for **YOLOv26**, tailored for deployment on **ESP32-P4** (via ESP-DL and ESP-PPQ).

### Key Features of YOLOv26
YOLOv26 represents an evolution in the YOLO architecture focusing on efficiency and end-to-end deployment.
- **NMS-Free Prediction**: Utilizes a dual-head architecture (One-to-Many for training signal, One-to-One for inference), eliminating the need for Non-Maximum Suppression (NMS) during inference.
- **RegMax=1**: Unlike YOLOv8 (RegMax=16) which uses Distribution Focal Loss (DFL), YOLOv26 uses direct regression (`RegMax=1`), reducing output channel complexity and post-processing overhead.
- **Efficient Attention**: Incorporates attention mechanisms (e.g., C2PSA) optimized for low-latency edge devices.

### Optimizer Choice: SGD vs MuoSGD
For this QAT pipeline, we have chosen **SGD (Stochastic Gradient Descent)** over **MuoSGD**.
- **Reason**: We involve only a small fine-tuning step on a model that is already pretrained and calibrated.
- **Simplicity**: Since the model starts in a good state, we do not need the additional complexity of MuoSGD (which is designed for more aggressive topology changes or from-scratch quantization training). Standard SGD is sufficient and effective for this task.

### QAT Workflow Specifics
This pipeline addresses specific challenges in quantizing YOLOv26 for embedded targets:
1.  **Custom Export Patches**: 
    - Patches `Attention` modules to use static reshaping (`view(-1, ...)`), ensuring compatibility with ESP-DL's static graph compiler.
    - Preserves all 6 output heads (3x Aux, 3x Main) during QAT to calculate accurate loss, while enabling `dynamic=False` export for final deployment.
2.  **Sensitive Layer Analysis**: Automatically identifies and disables quantization for the Auxiliary Branch to stabilize training.
3.  **PPQ Integration**: Uses the internal quantization pipeline (Simplify -> Fusion -> Parameter Quantize -> Calibration -> Finetuning) before starting the QAT loop.
4.  **Custom Validator**: A dedicated validator that mimics the Quantized Graph execution to report real on-target mAP metrics.


In [1]:
# Install Dependencies
%pip install -r requirements.txt

^C
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.0.1 -> 26.0
[notice] To update, run: python.exe -m pip install --upgrade pip


## 🧱 Step: Download & Prepare LEGO Minifigures Dataset
To showcase the **NMS-Free YOLO26n** capabilities, we are using the LEGO Minifigures dataset. 
This dataset is ideal because:
* **High Density:** Multiple small objects in close proximity.
* **Vibrant Colors:** Excellent for testing Int8 Quantization accuracy.
* **NMS-Free Test:** Directly regressing bounding boxes for similar-looking figures.

**Dataset Source:** [Lego Minifigures on Roboflow](https://universe.roboflow.com/hexhewwie/hex-lego)

In [ ]:
%pip install roboflow

from roboflow import Roboflow
rf = Roboflow(api_key="xxxxxxxxxxxxxx")
project = rf.workspace("hexhewwie").project("hex-lego")
version = project.version(3)
dataset = version.download("yolov5", location="lego_data")         


[notice] A new release of pip is available: 25.0.1 -> 26.0
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.
loading Roboflow workspace...
loading Roboflow project...


## Step: Fine-Tuning YOLO26n on LEGO Data
We will now perform transfer learning. We start with the pre-trained `yolo26n.pt` weights and fine-tune them on our LEGO dataset. 
* **Model:** YOLO26n (NMS-Free)
* **Epochs:** 50 (Adjustable)
* **Image Size:** 512 (Matching your ESP32-P4 optimization)

In [ ]:
import yaml

yaml_path = 'lego_data/data.yaml'

# 1. Read the current yaml
with open(yaml_path, 'r') as f:
    data = yaml.safe_load(f)

# 2. Fix the path logic
# We set 'path' to the absolute current directory or just '.'
data['path'] = 'lego_data' 
# Ensure the sub-paths don't repeat the folder name
data['train'] = 'train/images'
data['val'] = 'valid/images'
data['test'] = 'test/images'

# 3. Write it back
with open(yaml_path, 'w') as f:
    yaml.dump(data, f)

print("✅ data.yaml paths updated!")
print(f"New 'path' is: {data['path']}")

✅ data.yaml paths updated!
New 'path' is: lego_data


In [1]:
from ultralytics import YOLO
import os
# Disable Comet, WandB and other loggers to keep output clean
os.environ["COMET_MODE"] = "disabled"
os.environ["WANDB_MODE"] = "disabled"

# 1. Load the YOLO26n model
model = YOLO('yolo26n.pt') 

# 2. Start Fine-Tuning
# We use 'lego_data/data.yaml' which was created by the Roboflow download.
results = model.train(
    data="lego_data/data.yaml",   # Path to the LEGO data config
    epochs=40,                   # 10 epochs for fine-tuning
    imgsz=512,                   # Optimized size for ESP32-P4
    batch=48,                    # Adjust based on your GPU memory
    device=0,                    # Use '0' for GPU
    plots=True,                  # Generates training charts
    
    # --- Learning Rate Adjustments ---
    optimizer='MuSGD',
    lr0=0.001,                   # Initial learning rate (Standard fine-tuning)
    lrf=0.01,                    # Final learning rate (lr0 * lrf)
    warmup_epochs=2.0            # Slowly ramp up for 2 epochs to stabilize
)

print("✅ Training complete! Weights saved in 'runs/detect/trainx/weights/best.pt'")

New https://pypi.org/project/ultralytics/8.4.12 available  Update with 'pip install -U ultralytics'
Ultralytics 8.4.7  Python-3.9.21 torch-2.8.0+cu126 CUDA:0 (NVIDIA GeForce RTX 4060 Laptop GPU, 8188MiB)
engine\trainer: agnostic_nms=False, amp=True, angle=1.0, augment=False, auto_augment=randaugment, batch=48, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=lego_data/data.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=40, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=512, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.001, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolo26n.pt, momentum=0.937, mosaic=1.0, multi_scale=0.0, name=train7, nbs=64, nms

COMET ERROR: Failed to create Comet experiment, reason: ValueError('Comet.ml requires an API key. Please provide as the first argument to Experiment(api_key) or as an environment variable named COMET_API_KEY ')


WARNING Comet installed but not initialized correctly, not logging this run. Comet.ml requires an API key. Please provide as the first argument to Experiment(api_key) or as an environment variable named COMET_API_KEY 
Overriding model.yaml nc=80 with nc=28

                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      6640  ultralytics.nn.modules.block.C3k2            [32, 64, 1, False, 0.25]      
  3                  -1  1     36992  ultralytics.nn.modules.conv.Conv             [64, 64, 3, 2]                
  4                  -1  1     26080  ultralytics.nn.modules.block.C3k2            [64, 128, 1, False, 0.25]     
  5                  -1  1    147712  ultralytics.nn.modul

In [1]:
import shutil
import os
from glob import glob

# 1. Find the latest 'best.pt' weights from your training runs
# This looks in all 'train*' folders and picks the most recent one
weight_files = glob("runs/detect/train*/weights/best.pt")

if weight_files:
    # Get the most recent folder based on creation time
    latest_weights = max(weight_files, key=os.path.getctime)
    
    # 2. Define your destination name
    destination = "yolo26n_lego.pt"
    
    # 3. Copy and rename the file to your notebook root
    shutil.copy(latest_weights, destination)
    
    print(f"✅ Success! Your LEGO model is saved as: {os.path.abspath(destination)}")
else:
    print("❌ Error: Could not find 'best.pt'. Did the training finish successfully?")

✅ Success! Your LEGO model is saved as: c:\Users\orani\bilel\git_projects\yolo26\yolo26n_esp32_repo\esp-dl\examples\tutorial\how_to_quantize_model\quantize_yolo26\yolo26n_lego.pt


# YOLOv26 Quantization Aware Training (QAT)

In [1]:
# System Imports
import os
import sys
# Add pipeline source to path for local imports
sys.path.append('scripts')

import types
import torch
import esp_ppq.lib as PFL
from esp_ppq.executor import TorchExecutor
from esp_ppq.core import QuantizationVisibility, TargetPlatform
from esp_ppq.api import get_target_platform
from esp_ppq.api.interface import load_onnx_graph
from esp_ppq.quantization.optim import (
    QuantizeSimplifyPass, QuantizeFusionPass, ParameterQuantizePass,
    RuntimeCalibrationPass, PassiveParameterQuantizePass, QuantAlignmentPass
)
from ultralytics.data.utils import check_det_dataset


    ___________ ____        ____  ____  ____
   / ____/ ___// __ \      / __ \/ __ \/ __ \
  / __/  \__ \/ /_/ /_____/ /_/ / /_/ / / / /
 / /___ ___/ / ____/_____/ ____/ ____/ /_/ /
/_____//____/_/         /_/   /_/    \___\_\




In [17]:
# --- Important Configuration ---
IMG_SZ_I = 512   # any size
PLATFORM = "p4"  # p4 or s3
DATA_YAML_FILE_I = "lego_data/data.yaml" # coco.yaml Ultralytics default YAML file ,you can use your own
PROJECT_NAME = "lego"

In [18]:
# --- Configuration ---
class QATConfig:
    # Training Parameters
    EPOCHS = 20           # 640 img_sz need about 10 epochs
    BATCH_SIZE = 20       # on 8GB VRAM use 14 for 640 img_sz and 20 for 512 img_sz
    IMG_SZ = IMG_SZ_I
    DATA_FRACTION = 1.0   # Use 0.005 (0.5%) of dataset for fast debugging
    SEED = 1234
    DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
    
    # Optimizer
    OPTIMIZER_LR = 1e-6
    OPTIMIZER_MOMENTUM = 0.937
    OPTIMIZER_WEIGHT_DECAY = 5e-4
    
    # Data Settings
    DATA_YAML_FILE = DATA_YAML_FILE_I
    DATA_FALLBACK_PATH = "coco2017/images/train2017"
    CALIB_MAX_IMAGES = 8192
    CALIB_VALID_EXTENSIONS = ('.jpg', '.jpeg', '.png', '.bmp', '.webp')
    
    # Quantization Settings
    TARGET_PLATFORM = get_target_platform("esp32"+PLATFORM, 8)
    CALIB_STEPS = 32
    QUANT_CALIB_METHOD = "kl"
    QUANT_ALIGNMENT = "Align to Output"
    EXPORT_OPSET = 13
    EXPORT_DYNAMIC = False 
    
    # Loss Defaults (Standard YOLOv8/v10/v26 defaults)
    LOSS_DEFAULTS = {
        'box': 7.5, 'cls': 0.5, 'dfl': 1.5, 'pose': 12.0, 'kobj': 1.0,
        'label_smoothing': 0.0, 'nbs': 64, 'hsv_h': 0.015, 'hsv_s': 0.7,
        'hsv_v': 0.4, 'degrees': 0.0, 'translate': 0.1, 'scale': 0.5,
        'shear': 0.0, 'perspective': 0.0, 'flipud': 0.0, 'fliplr': 0.5,
        'mosaic': 1.0, 'mixup': 0.0, 'copy_paste': 0.0,
    }

    # Model Paths
    # Assuming current directory
    BASE_DIR = os.getcwd()
    MODEL_NAME = "yolo26n"+"_"+PROJECT_NAME 
    PT_FILE = f"{MODEL_NAME}.pt"
    ONNX_FILE = f"{MODEL_NAME}_train.onnx"
    
    # Derived Paths - Output Structure updated for GitHub workflow
    ESPDL_OUTPUT_DIR = os.path.join(BASE_DIR, "output", f"{PROJECT_NAME}_{IMG_SZ}_s8_{PLATFORM}")
    ONNX_PATH = os.path.join(ESPDL_OUTPUT_DIR, ONNX_FILE)
    
    # Plotting
    VAL_PLOT_MAX_BATCHES = 3

    # Validation Batch Size
    VAL_BATCH_SIZE = 24

In [4]:
# --- Virtual Module Injection ---
# This tricks Python into thinking 'config.py' exists and contains our QATConfig class.
# This allows 'trainer.py' and 'utils.py' to do 'from config import QATConfig'

if 'config' in sys.modules:
    # If it exists, update it
    sys.modules['config'].QATConfig = QATConfig
else:
    # Create a dummy module
    config_module = types.ModuleType('config')
    config_module.QATConfig = QATConfig
    sys.modules['config'] = config_module

# Must ensure output directory exists for config files early usage if any
if not os.path.exists(QATConfig.ESPDL_OUTPUT_DIR):
    os.makedirs(QATConfig.ESPDL_OUTPUT_DIR)

# Now we can safely import our local modules that depend on 'config'
from utils import seed_everything, register_mod_op, patch_v8_detection_loss, get_exclusive_ancestors
from dataset import get_calibration_loader, get_train_loader
from trainer import QATTrainer
from export import apply_export_patches, ESP_YOLO

# Set seeds for reproducibility
seed_everything(QATConfig.SEED)
register_mod_op()
patch_v8_detection_loss()
print("Virtual Config injected and Modules initialized.")

Registered 'Mod' handler for PPQ.
Virtual Config injected and Modules initialized.


### ESP-PPQ ONNX Compatibility Patch

In [5]:
from esp_ppq_patch import apply_esp_ppq_patches
apply_esp_ppq_patches()

Applying ESP-PPQ Runtime Patches...
  [x] Patched OnnxParser.refine_graph
  [x] Patched Backend: Slice
  [x] Patched Backend: Gather
ESP-PPQ Runtime Patches Applied Successfully.


## 1. Model Preparation & Export
We load the PyTorch checkpoint (`.pt`) and export it to ONNX. 
Critically, we apply `ESP_Attention` patches here to ensure the exported ONNX graph uses static reshaping, preventing runtime errors on the target device.

In [6]:
def extract_model_meta():
    """Dynamically extracts model metadata (NC, RegMax, etc.) from the PT checkpoint."""
    print(f"Loading {QATConfig.PT_FILE} to extract metadata...")
    # We use ESP_YOLO here just to inspect, but standard YOLO would work too for this part
    tmp_model = ESP_YOLO(QATConfig.PT_FILE)
    
    # Access the Detect Head (last layer)
    detect_head = tmp_model.model.model[-1]
    
    # Derive input channels (ch) from the first layer of cv2
    ch = [m[0].conv.in_channels for m in detect_head.cv2]
    
    meta = {
        'nc': detect_head.nc,
        'reg_max': detect_head.reg_max,
        'stride': detect_head.stride,
        'ch': ch
    }
    
    if isinstance(meta['stride'], torch.Tensor):
        meta['stride'] = meta['stride'].tolist()
        
    print(f"Extracted Metadata: NC={meta['nc']}, RegMax={meta['reg_max']}, Stride={meta['stride']}")
    return meta

def prepare_onnx():
    """Ensures the correct ONNX model exists with all patches applied."""
    try:
        # Load using ESP_YOLO to enforce custom export logic
        model = ESP_YOLO(QATConfig.PT_FILE)
        
        # Apply patches (Attention & Detect)
        apply_export_patches(model)
        
        # Export
        model.export(format="onnx", opset=QATConfig.EXPORT_OPSET, simplify=True, 
                        imgsz=QATConfig.IMG_SZ, dynamic=QATConfig.EXPORT_DYNAMIC)
        print("Export complete.")
            
    except Exception as e:
        print(f"Error exporting model: {e}")
        raise e

# Run Preparation
prepare_onnx()
model_meta = extract_model_meta()

Applying ESP-DL patches for export...
Patched 2 Attention modules.
Patched Detect module: <class 'ultralytics.nn.modules.head.Detect'>
Ultralytics 8.4.7  Python-3.9.21 torch-2.8.0+cu126 CPU (13th Gen Intel Core(TM) i7-13650HX)
>> Fuse method blocked! Keeping all heads.
YOLO26n summary (fused): 146 layers, 2,505,224 parameters, 0 gradients, 5.7 GFLOPs

PyTorch: starting from 'yolo26n_lego.pt' with input shape (16, 3, 512, 512) BCHW and output shape(s) ((16, 32, 64, 64), (16, 32, 32, 32), (16, 32, 16, 16), (16, 32, 64, 64), (16, 32, 32, 32), (16, 32, 16, 16)) (5.1 MB)

ONNX: starting export with onnx 1.17.0 opset 13...
ONNX: simplifying with onnxsim 0.4.36...
ONNX: export success  1.3s, saved as 'c:\Users\orani\bilel\git_projects\yolo26\yolo26n_esp32_repo\esp-dl\examples\tutorial\how_to_quantize_model\quantize_yolo26\output\lego_data/data_512_s8_p4\yolo26n_lego_train.onnx' (9.7 MB)

Export complete (2.3s)
Results saved to C:\Users\orani\bilel\git_projects\yolo26\yolo26n_esp32_repo\esp-dl

## 2. Quantizer Initialization
We load the ONNX graph and initialize the PPQ Quantizer.
We also perform a graph analysis to separate the **Auxiliary Branch** (used for training guidance) from the **Main Branch** (used for inference). We disable quantization on the Aux branch to ensure gradients flow correctly without noise.

In [7]:
# Load Graph
graph = load_onnx_graph(onnx_import_file=QATConfig.ONNX_PATH)

# Identify and Disable Aux Layer Quantization
output_names = list(graph.outputs.keys())
aux_ops = set()
if len(output_names) >= 6:
    # Assuming order: [one2many_p3, ... one2one_p3, ...]
    # Note: Custom exporter uses names one2many_p3 etc. which is explicitly handled here
    aux_outputs = output_names[0:3]
    main_outputs = output_names[3:6]
    print("Identifying auxiliary branch operators to disable quantization...")
    aux_ops = get_exclusive_ancestors(graph, aux_outputs, main_outputs)
    print(f"Found {len(aux_ops)} operators exclusive to auxiliary branch.")
else:
    print("WARNING: Graph output count < 6. Cannot separate aux/main branches.")

# Initialize Quantizer
quantizer = PFL.Quantizer(platform=QATConfig.TARGET_PLATFORM, graph=graph)
dispatching_table = PFL.Dispatcher(graph=graph, method="conservative").dispatch(
    quantizer.quant_operation_types
)

# Enforce FP32 for Aux ops and defaults
for opname, platform in dispatching_table.items():
    if platform == TargetPlatform.UNSPECIFIED:
        dispatching_table[opname] = TargetPlatform(quantizer.target_platform)
        
for op in aux_ops:
    if op.name in dispatching_table:
        dispatching_table[op.name] = TargetPlatform.FP32

# Apply Quantization
for op in graph.operations.values():
    quantizer.quantize_operation(op_name=op.name, platform=dispatching_table[op.name])

Identifying auxiliary branch operators to disable quantization...
Found 45 operators exclusive to auxiliary branch.


## 3. Calibration
Before training, we must calibrate the quantized parameters (scale/offset) using a representative dataset. This initializes the network in a good state.

In [8]:
# Data Loaders
data_cfg = check_det_dataset(QATConfig.DATA_YAML_FILE)
cali_loader = get_calibration_loader(data_cfg)
train_loader = get_train_loader(data_cfg)

# Tracing
executor = TorchExecutor(graph=graph)
dummy_input = torch.zeros([1, 3, QATConfig.IMG_SZ, QATConfig.IMG_SZ]).to(QATConfig.DEVICE)
executor.tracing_operation_meta(inputs=dummy_input)

# Calibration Pipeline
print("Running Calibration Pipeline...")
pipeline = PFL.Pipeline([
    QuantizeSimplifyPass(),
    QuantizeFusionPass(activation_type=quantizer.activation_fusion_types),
    ParameterQuantizePass(),
    RuntimeCalibrationPass(method=QATConfig.QUANT_CALIB_METHOD),
    PassiveParameterQuantizePass(clip_visiblity=QuantizationVisibility.EXPORT_WHEN_ACTIVE),
    QuantAlignmentPass(elementwise_alignment=QATConfig.QUANT_ALIGNMENT),
])

pipeline.optimize(
    calib_steps=QATConfig.CALIB_STEPS,
    collate_fn=(lambda x: x.type(torch.float).to(QATConfig.DEVICE)),
    graph=graph,
    dataloader=cali_loader,
    executor=executor,
)

Using dataset at: C:\Users\orani\bilel\git_projects\yolo26\yolo26n_esp32_repo\esp-dl\examples\tutorial\how_to_quantize_model\quantize_yolo26\lego_data\train\images
Fast image access  (ping: 0.00.0 ms, read: 431.9217.9 MB/s, size: 27.6 KB)
Scanning C:\Users\orani\bilel\git_projects\yolo26\yolo26n_esp32_repo\esp-dl\examples\tutorial\how_to_quantize_model\quantize_yolo26\lego_data\train\labels.cache... 7408 images, 3694 backgrounds, 0 corrupt: 100% ━━━━━━━━━━━━ 7408/7408  0.0s
[WARNING][PPQ][2026-02-07 05:15:32]:  Unexpected input value of operation /model.11/Resize, recieving "None" at its input 1
[WARNING][PPQ][2026-02-07 05:15:32]:  Unexpected input value of operation /model.14/Resize, recieving "None" at its input 1
Running Calibration Pipeline...
[05:15:32] PPQ Quantize Simplify Pass Running ...         Finished.
[05:15:32] PPQ Quantization Fusion Pass Running ...       Finished.
[05:15:32] PPQ Parameter Quantization Pass Running ...    Finished.
[05:15:32] PPQ Runtime Calibration Pa

Calibration Progress(Phase 2): 100%|██████████| 32/32 [00:06<00:00,  4.79it/s]


Finished.
[05:15:47] PPQ Passive Parameter Quantization Running ... Finished.
[05:15:47] PPQ Quantization Alignment Pass Running ...    Finished.


### Baseline Validation (Check PTQ Accuracy)

In [9]:
# Initialize the trainer just for evaluation
print("Initializing Trainer for Baseline Check...")
trainer = QATTrainer(graph=graph, model_meta=model_meta, device=QATConfig.DEVICE)

# Run validation on the graph in its current state (after Calibration/PTQ, before Training)
print("Running Baseline Validation on Quantized Graph...")
ptq_mAP = trainer.eval()

print(f"\n--- Baseline Results ---")
print(f"PTQ mAP50-95: {ptq_mAP:.3f}")
print(f"Target: This serves as the baseline. QAT will now attempt to improve this score.")

Initializing Trainer for Baseline Check...
Loading YOLOv26n model in Trainer to access correct Loss function...
Initializing Persistent Validator (reusing dataloader)...
Running Baseline Validation on Quantized Graph...
Ultralytics 8.4.7  Python-3.9.21 torch-2.8.0+cu126 CUDA:0 (NVIDIA GeForce RTX 4060 Laptop GPU, 8188MiB)
val: Fast image access  (ping: 0.00.0 ms, read: 403.7246.3 MB/s, size: 24.8 KB)
val: Scanning C:\Users\orani\bilel\git_projects\yolo26\yolo26n_esp32_repo\esp-dl\examples\tutorial\how_to_quantize_model\quantize_yolo26\lego_data\valid\labels.cache... 628 images, 302 backgrounds, 0 corrupt: 100% ━━━━━━━━━━━━ 628/628  0.0s
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 27/27 1.8it/s 14.7s0.5s
                   all        628       1292      0.801      0.696      0.798      0.624

--- Baseline Results ---
PTQ mAP50-95: 0.624
Target: This serves as the baseline. QAT will now attempt to improve this score.


## 4. QAT Training Loop
We now fine-tune the quantized model. The `QATTrainer` handles the forward pass through the quantized graph (simulated by PPQ) and the backward pass to update weights.

Note: The validation step uses our `QuantizedModelValidator` which manually decodes the raw graph outputs using the metadata extracted earlier (`NC`, `RegMax`, `Estride`).

In [10]:
print("Starting QAT Training...")

if not os.path.exists(QATConfig.ESPDL_OUTPUT_DIR):
    os.makedirs(QATConfig.ESPDL_OUTPUT_DIR)

best_mAP = 0
for epoch in range(QATConfig.EPOCHS):
    print(f"\n--- Epoch {epoch+1}/{QATConfig.EPOCHS} ---")
    
    # Train Epoch
    trainer.epoch(train_loader)
    
    # Validate
    current_mAP = trainer.eval()
    print(f"Epoch: {epoch+1}, mAP50-95: {current_mAP:.3f}")
    
    if current_mAP > best_mAP:
        best_mAP = current_mAP
        print(f"New best mAP! Saving to {QATConfig.ESPDL_OUTPUT_DIR}...")
        
        # Save Native Graph using the new helper method
        trainer.save_graph(os.path.join(QATConfig.ESPDL_OUTPUT_DIR, "Best_yolo26n.native"))

Starting QAT Training...

--- Epoch 1/20 ---


Epoch 0: 100%|██████████| 371/371 [06:14<00:00,  1.01s/it, loss=5.8876] 

Epoch Loss: 22.4570
Ultralytics 8.4.7  Python-3.9.21 torch-2.8.0+cu126 CUDA:0 (NVIDIA GeForce RTX 4060 Laptop GPU, 8188MiB)


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 27/27 3.5it/s 7.7s0.3s
                   all        628       1292      0.871      0.756      0.831      0.676
Epoch: 1, mAP50-95: 0.676
New best mAP! Saving to c:\Users\orani\bilel\git_projects\yolo26\yolo26n_esp32_repo\esp-dl\examples\tutorial\how_to_quantize_model\quantize_yolo26\output\lego_data/data_512_s8_p4...

--- Epoch 2/20 ---


Epoch 1: 100%|██████████| 371/371 [06:13<00:00,  1.01s/it, loss=6.1593] 

...

Epoch 12: 100%|██████████| 371/371 [06:31<00:00,  1.05s/it, loss=6.0918] 

Epoch Loss: 18.7304
Ultralytics 8.4.7  Python-3.9.21 torch-2.8.0+cu126 CUDA:0 (NVIDIA GeForce RTX 4060 Laptop GPU, 8188MiB)


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 27/27 2.1it/s 13.1s0.5s
                   all        628       1292      0.873      0.769      0.848        0.7
Epoch: 13, mAP50-95: 0.700

--- Epoch 14/20 ---


Epoch 13:  41%|████      | 151/371 [02:27<03:34,  1.03it/s, loss=22.2825]


KeyboardInterrupt: 

In [11]:
# Load Best Graph back into memory at the end
print("Reloading Best Graph...")
trainer.load_graph(os.path.join(QATConfig.ESPDL_OUTPUT_DIR, "Best_yolo26n.native"))
# Update global graph 
graph = trainer.graph 
print("Best Graph Reloaded.")

Reloading Best Graph...
Best Graph Reloaded.


In [ ]:
import esp_ppq.lib as PFL
from config import QATConfig
import os
from esp_ppq.api import load_native_graph
from esp_ppq.IR import BaseGraph  

# --- Helper Function: Robust Graph Pruning ---
def prune_graph_safely(graph: BaseGraph) -> BaseGraph:
    """
    Robust pruning function for ESP-PPQ graphs.
    Safely removes disconnected operations and unused variables.
    """
    print("Starting Safe Pruning Procedure...")
    round_count = 0
    while True:
        this_round_op_removed = 0
        this_round_var_removed = 0
        
        # A. Find Dead Ops
        dead_ops = []
        for op in list(graph.operations.values()):
            is_output = any(var.name in graph.outputs for var in op.outputs)
            has_consumers = any(len(var.dest_ops) > 0 for var in op.outputs)
            
            if not is_output and not has_consumers:
                dead_ops.append(op)
        
        # Remove Dead Ops Safely
        for op in dead_ops:
            for var in list(op.inputs):
                 op.inputs.remove(var)
                 if op in var.dest_ops:
                     var.dest_ops.remove(op)
            graph.remove_operation(op, keep_coherence=False)
            this_round_op_removed += 1
            
        # B. Find Dead Variables
        dead_vars = []
        for var in list(graph.variables.values()):
            is_input = var.name in graph.inputs
            is_output = var.name in graph.outputs
            if is_input or is_output: continue
            
            if len(var.dest_ops) == 0:
                dead_vars.append(var)
                 
        # Remove Dead Variables
        for var in dead_vars:
            if var.name in graph.variables:
                graph.variables.pop(var.name)
                this_round_var_removed += 1
        
        round_count += 1
        if this_round_op_removed == 0 and this_round_var_removed == 0:
            break
            
    print(f"Pruning Finished. Total Rounds: {round_count}")
    return graph

# --- MAIN LOGIC: Split Outputs at Concat Source ---

print("Reloading Best Graph...")
native_graph_path = os.path.join(QATConfig.ESPDL_OUTPUT_DIR, "Best_yolo26n.native")
graph = load_native_graph(import_file=native_graph_path)

print("Preparing Graph for Inference...")

# 1. Remove Aux Heads First (Cleanup)
output_names = list(graph.outputs.keys())
if len(output_names) >= 6:
    aux_heads = output_names[0:3] 
    print(f"Removing Aux Heads: {aux_heads}")
    for name in aux_heads:
        if name in graph.outputs:
            graph.outputs.pop(name)
    prune_graph_safely(graph)

# 2. Apply Splitting Strategy
targets = ["one2one_p3", "one2one_p4", "one2one_p5"]
collected_outputs = {}

for target_name in targets:
    if target_name in graph.outputs:
        original_output_var = graph.variables[target_name]
        producer = original_output_var.source_op 
        
        if producer and producer.type == "Concat":
            print(f"Splitting {target_name} at Source Concat ({producer.name})...")
            
            box_var = None
            cls_var = None
            
            # Inspect Concat inputs to find Box(4) and Cls(80)
            for input_var in producer.inputs:
                dims = input_var.shape
                if dims is not None:
                    if 4 in dims: box_var = input_var
                    elif model_meta['nc'] in dims: cls_var = input_var
            
            if box_var and cls_var:
                # Rename Scheme: Suffix per Request
                pair_config = [
                    (box_var, f"{target_name}_box"),  # e.g. one2one_p3_box
                    (cls_var, f"{target_name}_cls")   # e.g. one2one_p3_cls
                ]
                
                for var, new_name in pair_config:
                    old_name = var.name
                    
                    # Robust Renaming (Modify Private + Registry)
                    if old_name in graph.variables:
                        graph.variables.pop(old_name)
                    
                    var._name = new_name
                    graph.variables[new_name] = var
                    
                    collected_outputs[new_name] = var
                
                # Update Graph Outputs: Remove old name
                graph.outputs.pop(target_name)
                
                # Properly Remove Concat Op
                graph.remove_operation(producer, keep_coherence=False)
                # Unlink inputs
                for var in producer.inputs:
                    if producer in var.dest_ops:
                        var.dest_ops.remove(producer)

                print(f"  -> Created {pair_config[0][1]} and {pair_config[1][1]}")
            else:
                print(f"ERROR: Shape mismatch for {target_name}")
        else:
             print(f"WARNING: Source for {target_name} is not Concat.")

# 3. Register New Outputs in Strict Order (Box Group then Cls Group)
final_output_list = [
    "one2one_p3_box", "one2one_p4_box", "one2one_p5_box",
    "one2one_p3_cls", "one2one_p4_cls", "one2one_p5_cls"
]

print("Updating Graph Output Order...")
graph.outputs.clear() # Enforce strict order
count_added = 0
for name in final_output_list:
    if name in collected_outputs:
        graph.outputs[name] = collected_outputs[name]
        count_added += 1

print(f"Registered {count_added} split outputs.")

# 4. Final Prune
prune_graph_safely(graph)

print(f"Final Graph Outputs: {list(graph.outputs.keys())}") 

# 5. Export
inference_export_path = os.path.join(QATConfig.ESPDL_OUTPUT_DIR, f"{QATConfig.MODEL_NAME}_{QATConfig.IMG_SZ}_s8_{PLATFORM}.espdl")
print(f"Exporting Split Inference Model to {inference_export_path}...")

exporter = PFL.Exporter(platform=QATConfig.TARGET_PLATFORM)
exporter.export(inference_export_path, graph=graph)
print("Export Done!")

Reloading Best Graph...
Preparing Graph for Inference...
Removing Aux Heads: ['one2many_p3', 'one2many_p4', 'one2many_p5']
Starting Safe Pruning Procedure...
Pruning Finished. Total Rounds: 11
Splitting one2one_p3 at Source Concat (/model.23/Concat_3)...
  -> Created one2one_p3_box and one2one_p3_cls
Splitting one2one_p4 at Source Concat (/model.23/Concat_4)...
  -> Created one2one_p4_box and one2one_p4_cls
Splitting one2one_p5 at Source Concat (/model.23/Concat_5)...
  -> Created one2one_p5_box and one2one_p5_cls
Updating Graph Output Order...
Registered 6 split outputs.
Starting Safe Pruning Procedure...
Pruning Finished. Total Rounds: 2
Final Graph Outputs: ['one2one_p3_box', 'one2one_p4_box', 'one2one_p5_box', 'one2one_p3_cls', 'one2one_p4_cls', 'one2one_p5_cls']
Exporting Split Inference Model to c:\Users\orani\bilel\git_projects\yolo26\yolo26n_esp32_repo\esp-dl\examples\tutorial\how_to_quantize_model\quantize_yolo26\output\lego_data/data_512_s8_p4\yolo26n_lego_512_s8_p4.espdl...
